Estimación del ingreso trimestral por persona

In [ ]:
##LIBRERIAS
librerias <- c("tidyverse",
               "zip",
               "readr",
               "srvyr")

instalar <- rownames(installed.packages())
instalar_librerias <- setdiff(librerias, instalar)

if (length(instalar_librerias) > 0) {
  install.packages(instalar_librerias)
}

lapply(librerias, library, character.only = TRUE)

In [3]:
#CREAR DIRECTORIO DE TRABAJO
dir.create("ingreso_enigh", showWarnings = FALSE)
setwd("ingreso_enigh")

In [ ]:
##CARGAR LA BASE DE INGRESOS DE LA ENCUESTA NACIONAL DE INGRESOS Y GASTOS DE LOS HOGARES 2022
url <- "https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_ingresos_csv.zip"
options(timeout = 500)
download.file(url, 'enigh2022_ns_ingresos_csv.zip', mode = "wb")
unzip('enigh2022_ns_ingresos_csv.zip', files = "ingresos.csv")
ingreso_enigh <- read_csv("ingresos.csv", show_col_types = FALSE)|>
mutate(cve_ent = substr(folioviv, 1,2))|>
select(folioviv,foliohog, numren, ing_tri)%>%
group_by(folioviv, foliohog, numren)%>%
summarise(ing_tri = sum(ing_tri, na.rm = TRUE))

`summarise()` has grouped output by 'folioviv', 'foliohog'. You can override
using the `.groups` argument.


In [ ]:
##DESCARGAR LA BASE DE POBLACIÓN DE LA ENCUESTA NACIONAL DE INGRESOS Y GASTOS DE LOS HOGARES 2022
url<- "https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_poblacion_csv.zip"
options(timeout = 500)
download.file(url, 'enigh2022_ns_poblacion_csv.zip', mode = "wb")
unzip('enigh2022_ns_poblacion_csv.zip', files = "poblacion.csv")
poblacion_enigh<-read_csv("poblacion.csv", show_col_types = FALSE)|>
#OBTENER CLAVE DE ENTIDAD
mutate(cve_ent = substr(folioviv, 1,2))|>
#FILTRADO PARA LA CIUDAD DE MÉXICO
filter(cve_ent == "09")|> 
#SELECCIÓN DE CAMPOS DE INTERÉS
select(folioviv, foliohog, numren, nivelaprob, gradoaprob,upm, factor, est_dis, edad, cve_ent)|> 
mutate(gradoaprob = as.numeric(gradoaprob))|>
#UNIR CON TABLA DE INGRESOS
left_join(ingreso_enigh,by = c("folioviv","foliohog","numren"))|>
##ESTABLECER LOS AÑOS DE ESCOLARIDAD
mutate(anos_escolaridad = case_when (nivelaprob %in% c("1", "0") ~ 0,
                        #PRIMARIA
                        nivelaprob=="2" ~ gradoaprob, 
                        #SECUNDARIA
                        nivelaprob=="3" ~ gradoaprob+6, 
                        #PREPARATORIA
                        nivelaprob=="4" ~ gradoaprob+9, 
                        #LICENCIATURA O EQUIVALENTE
                        nivelaprob %in% c("5","6","7") ~ gradoaprob+12, 
                        #POSGRADO
                        nivelaprob %in% c("8","9") ~ gradoaprob+17), 
        #INGRESO MENSUAL
        ing_mensual = ing_tri/3)


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


In [6]:
##ESTABLECER EL DISEÑO MUESTRAL
dm<-poblacion_enigh|>
as_survey_design(weights = factor,
                id=upm,
                strata=est_dis)

In [7]:
##APROXIMACIÓN DEL INGRESO MENSUAL DE ACUEERDO AL GRADO DE ESCOLARIDAD
graproes<-dm|>
group_by(anos_escolaridad)|>
summarise(igproes=survey_mean(ing_mensual,na.rm = TRUE, vartype = "cv"))
graproes

anos_escolaridad,igproes,igproes_cv
<dbl>,<dbl>,<dbl>
0,1772.8630,0.09573637
1,1697.9391,0.22483675
2,2189.7338,0.11842374
3,2848.3375,0.11748765
4,2222.0659,0.13863855
5,2708.7470,0.16479705
6,4952.2518,0.04269781
7,2449.5479,0.14418367
8,3536.9337,0.08650728
